In [1]:
using LowLevelFEM

In [2]:
gmsh.initialize()
gmsh.open("truss1D.geo")

Info    : Reading 'truss1D.geo'...
Info    : Meshing 1D...
Info    : Meshing curve 1 (Line)
Info    : Done meshing 1D (Wall 0.000193351s, CPU 0.000182s)
Info    : 3 nodes 4 elements
Info    : Done reading 'truss1D.geo'


In [3]:
openPreProcessor()

-------------------------------------------------------
Version       : 4.15.0-git
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib tinyobjloader
FLTK version  : 1.3.8
OCC version   : 7.9.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


Fontconfig warning: using without calling FcInit()


In [4]:
fx_val = getParameter("fx")
F_val = getParameter("F")

10.0

In [5]:
mat = Material("truss", E=2e5)
prob = Problem([mat], type=:Truss, bandwidth=:Hilbert)

Info    : Hilbert renumbering...
Info    : Done Hilbert renumbering


Problem("truss1D", :Truss, 3, 3, Material[Material("truss", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 3, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [6]:
supp = displacementConstraint("left", ux=0)
suppYZ = displacementConstraint("truss", uy=0, uz=0)
F = load("right", fx=F_val)
fx = load("truss", fx=fx_val)

BoundaryCondition("truss", nothing, nothing, nothing, nothing, nothing, nothing, 1.0, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [7]:
K = stiffnessMatrix(prob)
K[:, :]

9×9 SparseArrays.SparseMatrixCSC{Float64, Int64} with 7 stored entries:
  4000.0   ⋅    ⋅   -4000.0   ⋅    ⋅        ⋅    ⋅    ⋅ 
      ⋅    ⋅    ⋅        ⋅    ⋅    ⋅        ⋅    ⋅    ⋅ 
      ⋅    ⋅    ⋅        ⋅    ⋅    ⋅        ⋅    ⋅    ⋅ 
 -4000.0   ⋅    ⋅    8000.0   ⋅    ⋅   -4000.0   ⋅    ⋅ 
      ⋅    ⋅    ⋅        ⋅    ⋅    ⋅        ⋅    ⋅    ⋅ 
      ⋅    ⋅    ⋅        ⋅    ⋅    ⋅        ⋅    ⋅    ⋅ 
      ⋅    ⋅    ⋅   -4000.0   ⋅    ⋅    4000.0   ⋅    ⋅ 
      ⋅    ⋅    ⋅        ⋅    ⋅    ⋅        ⋅    ⋅    ⋅ 
      ⋅    ⋅    ⋅        ⋅    ⋅    ⋅        ⋅    ⋅    ⋅ 

In [8]:
free = freeDoFs(prob, [suppYZ])
K[free, free]

3×3 SparseArrays.SparseMatrixCSC{Float64, Int64} with 7 stored entries:
  8000.0  -4000.0  -4000.0
 -4000.0   4000.0       ⋅ 
 -4000.0       ⋅    4000.0

In [9]:
cond = freeDoFs(prob, [suppYZ, supp])
K[cond, cond]

2×2 SparseArrays.SparseMatrixCSC{Float64, Int64} with 4 stored entries:
  8000.0  -4000.0
 -4000.0   4000.0

In [10]:
f = loadVector(prob, [F, fx])
DoFs(f)[:]

9-element Vector{Float64}:
 25.0
  0.0
  0.0
 50.0
  0.0
  0.0
 35.0
  0.0
  0.0

In [11]:
DoFs(f)[free]

3-element Vector{Float64}:
 50.0
 25.0
 35.0

In [12]:
DoFs(f)[cond]

2-element Vector{Float64}:
 50.0
 35.0

In [13]:
q = solveDisplacement(K, f, support=[suppYZ, supp])
DoFs(q)[free]

3-element Vector{Float64}:
 0.021249999999999998
 0.0
 0.03

In [14]:
q0 = showDoFResults(q, name="elmozdulás")
plotOnPath("truss", q0)

2

In [15]:
N = solveAxialForce(q)
N.A

2-element Vector{Matrix{Float64}}:
 [84.99999999999999; 84.99999999999999;;]
 [35.000000000000014; 35.000000000000014;;]

In [16]:
N0 = showElementResults(N, name="rúderő")
plotOnPath("truss", N0)

4

In [17]:
fix = constrainedDoFs(prob, [supp])

FT = K * q - f
DoFs(FT)[fix]

1-element Vector{Float64}:
 -109.99999999999999

In [18]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.15.0-git
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib tinyobjloader
FLTK version  : 1.3.8
OCC version   : 7.9.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [19]:
gmsh.finalize()